In [ ]:
import tensorflow as tf
import keras
from keras import layers
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix


from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Constantes
IS_TO_FIND_BEST_HYPERPARAMS = True
IS_TO_TRAIN = True
IS_TO_REPLICATE = True
BATCH_SIZE = 64
IMG_HEIGHT = 224    
IMG_WIDTH = 224

DATASET_TRAIN_PATH = "../cats_and_dogs/train"  # ajustar consoante a localização do dataset
           
DATASET_VAL_PATH = "../cats_and_dogs/validation"  # ajustar consoante a localização do dataset
NUM_CLASSES=2
SEED = None if IS_TO_TRAIN else 12345
MAX_EPOCHS = 50
WEIGHTS_FILENAME = "models/best_binary_model.weights.h5"
HYPERPARAMS_FILENAME = "models/best_binary_model.hyperparams.json"

In [20]:
# ficheiro onde serão guardados os pesos do "melhor modelo" - ajustar o caminho
BEST_MODEL_PATH = "tmp/best_model.weights.h5"

# callback para guardar o melhor modelo
BEST_MODEL_CHECKPOINT = keras.callbacks.ModelCheckpoint(
    filepath=BEST_MODEL_PATH,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# callback para parar o treino caso não se verifiquem melhorias na loss
EARLY_STOPPING = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5)

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [41]:

DATASET_PATH = "../cats_and_dogs"
SEED = 7654321  # semente para o split validação/teste com melhor distribuição


train_ds = keras.utils.image_dataset_from_directory(
    DATASET_PATH + '/train',
    labels='inferred',
    label_mode='binary',
    seed=SEED,
    batch_size=BATCH_SIZE
)

val_ds, test_ds = keras.utils.image_dataset_from_directory(
    DATASET_PATH + '/validation',
    labels='inferred',
    label_mode='binary',
    validation_split=0.5,
    subset="both",
    seed=SEED,
    batch_size=BATCH_SIZE
)

# as labels foram inferidas a partir dos nomes dos diretórios
labels = train_ds.class_names
print("Encontradas as classes: ", labels)

# carregar os datasets em memória - uma vez carregados, a ordem dos batches já não muda
train_ds = train_ds.cache()
val_ds = val_ds.cache()
test_ds = test_ds.cache()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xba in position 38: invalid start byte

In [21]:
initializer = keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=SEED)

catOrDog_model = keras.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.3),
    layers.Conv2D(16, 5, padding='same', activation='relu', kernel_initializer=initializer),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu', kernel_initializer=initializer),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer=initializer),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu', kernel_initializer=initializer),
    layers.MaxPooling2D(pool_size=4),
    layers.Flatten(),
    layers.Dropout(0.25),
    layers.Dense(3135, activation='relu', kernel_initializer=initializer),
    layers.Dropout(0.25),
    layers.Dense(1500, activation='relu', kernel_initializer=initializer),
    layers.Dense(NUM_CLASSES, activation="sigmoid")
])

# definição do algoritmo de otimização e da função de perda (loss)
catOrDog_model.compile(optimizer='adam',
                     loss="categorical_crossentropy",
                     metrics=['accuracy'])

# sumário do modelo
catOrDog_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_2 (RandomFlip)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_2               │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_2 (RandomZoom)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 224, 224, 16)   │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 112, 112, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 112, 112, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 56, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 56, 56, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 3135)           │    19,665,855 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 3135)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1500)           │     4,704,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 2)              │         3,002 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,471,065 (93.35 MB)

 Trainable params: 24,471,065 (93.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:


if IS_TO_FIND_BEST_HYPERPARAMS:
    SEED = 12345
    # Function to create model, required for KerasClassifier
    def create_model(learning_rate):
        global catOrDog_model

        catOrDog_model.compile(
            loss='binary_crossentropy',
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            metrics=['accuracy']
        )

        return catOrDog_model

    # Create the KerasClassifier
    model = KerasClassifier(model=create_model, verbose=0)

    # Define the grid search parameters
    param_grid = {
        'batch_size': [64, 128, 256, 512, 1024, 2048],
        'epochs': [MAX_EPOCHS],
        'model__learning_rate': np.linspace(0.01, 0.05, 0.09)
    }

    # Create GridSearchCV
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, verbose=0)

    grid_result = grid.fit(x_train, y_train)

    best_params = grid_result.best_params_

    # Save the best parameters to a JSON file
    with open(HYPERPARAMS_FILENAME, 'w') as json_file:
        json.dump(grid_result.best_params_, json_file, indent=4)

    print(f"\n\n\nBest model hyper parameters: {json.dumps(grid_result.best_params_, indent=4)}")

else:
    with open(HYPERPARAMS_FILENAME, 'r') as json_file:
        best_params = json.load(json_file)
        print(f"Best model hyper parameters from file: {json.dumps(best_params, indent=4)}")

NameError: name 'KerasClassifier' is not defined